# Vanilla LSTM - with One Hidden Layer

In [2]:
import pandas as pd
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
import numpy as np
from sklearn.model_selection import train_test_split

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
# univariate data preparation
from numpy import array
 
# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-1:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)
 
# define input sequence
raw_seq = [10, 20, 30, 40, 50, 60, 70, 80, 90]

# choose a number of time steps
n_steps = 3
# split into samples
X, y = split_sequence(raw_seq, n_steps)
# summarize the data
for i in range(len(X)):
	print(X[i], y[i])

[10 20 30] 40
[20 30 40] 50
[30 40 50] 60
[40 50 60] 70
[50 60 70] 80
[60 70 80] 90


In [4]:
# define input sequence
raw_seq = [10, 20, 30, 40, 50, 60, 70, 80, 90]
# choose a number of time steps
n_steps = 3
# split into samples
X, y = split_sequence(raw_seq, n_steps)
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))



# define model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [5]:
# fit model
model.fit(X, y, epochs=200, verbose=0)
# demonstrate prediction
x_input = array([70, 80, 90])
x_input = x_input.reshape((1, n_steps, n_features))
x_input

array([[[70],
        [80],
        [90]]])

In [6]:
x_input = x_input.astype(np.float32)

yhat1 = model.predict(x_input)
print(yhat1)

[[102.39444]]


# Stacked LSTM - with Multiple Hidden Layers

In [7]:
# define model
model2 = Sequential()
model2.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(n_steps, n_features)))
model2.add(LSTM(50, activation='relu'))
model2.add(Dense(1))
model2.compile(optimizer='adam', loss='mse')

In [8]:
# fit model
model2.fit(X, y, epochs=200, verbose=0)
# demonstrate prediction
x_input = array([70, 80, 90])
x_input = x_input.reshape((1, n_steps, n_features))

In [9]:
x_input = x_input.astype(np.float32)
yhat2 = model2.predict(x_input)
print(yhat2)

[[102.293945]]


# Bidirectional LSTM - learn both forward and backwards and concatenate both interpretations

In [10]:
# define model
model3 = Sequential()
model3.add(Bidirectional(LSTM(50, activation='relu'), input_shape=(n_steps, n_features)))
model3.add(Dense(1))
model3.compile(optimizer='adam', loss='mse')

In [11]:
# fit model
model3.fit(X, y, epochs=200, verbose=0)
# demonstrate prediction
x_input = array([70, 80, 90])
x_input = x_input.reshape((1, n_steps, n_features))

In [12]:
x_input = x_input.astype(np.float32)
yhat3 = model3.predict(x_input)
print(yhat3)

[[101.79405]]


# Summary of Each Model

In [13]:
# summarize the data
for i in range(len(X)):
	print(X[i], y[i])

[[10]
 [20]
 [30]] 40
[[20]
 [30]
 [40]] 50
[[30]
 [40]
 [50]] 60
[[40]
 [50]
 [60]] 70
[[50]
 [60]
 [70]] 80
[[60]
 [70]
 [80]] 90


In [14]:
x_input

array([[[70.],
        [80.],
        [90.]]], dtype=float32)

In [15]:
print(yhat1,yhat2,yhat3)  #Bidirectional learning performs the best

[[102.39444]] [[102.293945]] [[101.79405]]


# Time Series Forecasting

In [16]:
# Download the data from https://datahack.analyticsvidhya.com/contest/genpact-machine-learning-hackathon-1/


data = pd.read_csv(r'D:\Dropbox (eClerx Services Ltd.)\Profile\Downloads\train.csv')
data.tail()

,id,week,center_id,meal_id,checkout_price,base_price,emailer_for_promotion,homepage_featured,num_orders
456543,1271326,145,61,1543,484.09,484.09,0,0,68
456544,1062036,145,61,2304,482.09,482.09,0,0,42
456545,1110849,145,61,2664,237.68,321.07,0,0,501
456546,1147725,145,61,2569,243.50,313.34,0,0,729
456547,1361984,145,61,2490,292.03,290.03,0,0,162


In [17]:
data['num_orders'].describe()

count    456548.000000
mean        261.872760
std         395.922798
min          13.000000
25%          54.000000
50%         136.000000
75%         324.000000
max       24299.000000
Name: num_orders, dtype: float64

In [44]:
#train test split
x = data['num_orders']

# split into samples
X, y = split_sequence(x, n_steps)

xtrain,xtest,ytrain,ytest = train_test_split(X,y,test_size=0.25)

xtrain = np.reshape(xtrain, (xtrain.shape[0], 1, xtrain.shape[1]))
xtest = np.reshape(xtest, (xtest.shape[0], 1, xtest.shape[1]))

### Sequence data may be a 1D or 2D matrix of numbers but LSTM input layer requires a 3D format.

In [61]:
#Samples. One sequence is one sample. A batch is comprised of one or more samples.
#Time Steps. One time step is one point of observation in the sample.
#Features. One feature is one observation at a time step.

In [66]:
X.shape

(456543, 5, 1)

In [67]:
y.shape

(456543,)

In [63]:
xtrain.shape

(342407, 1, 5)

In [64]:
ytrain.shape

(342407,)

In [73]:
n_steps = 5
n_features = 1

# reshape from [samples, timesteps] into [samples, timesteps, features]
X = X.reshape((1,n_steps,n_features))

# define model
model4 = Sequential()
model4.add(Bidirectional(LSTM(50, activation='relu'), input_shape=(n_steps, n_features)))
model4.add(Dense(1))

# Compiling the model
model4.compile(optimizer='adam', loss='mse')

ValueError: cannot reshape array of size 2282715 into shape (1,5,1)

In [68]:
# Fitting the model
model4.fit(xtrain,ytrain,epochs= 200,verbose=0)

ValueError: Error when checking input: expected bidirectional_9_input to have shape (5, 1) but got array with shape (1, 5)

In [ ]:
xtest = xtest.astype(np.float32)
pred = model4.predict(xtest)